In [1]:
from PyPESTO.FRP import create_FRP2_v1
%load_ext autoreload
%autoreload 2

The tests rely on tellurium to construct the models
Since tellurium is not installed the tests can't be run
If you want to run the tests, pip install tellurium first


In [2]:
# sbml_model_filepath = '/SBML/PyPESTO/FRP/sbml_model.xml'
amici_model, sbml_model_filepath = create_FRP2_v1.load_amici_from_sbml()

Creating SBML model (FRP2 v1)
Creating species.
Generating parameters.
Creating reactions
Importing AMICI model from SBML


In [3]:
kpAA_true = create_FRP2_v1.kpAA_true = 4.0
kpAB_true = create_FRP2_v1.kpAB_true = 0.2
kpBA_true = create_FRP2_v1.kpBA_true = 15.0
kpBB_true = create_FRP2_v1.kpBB_true = 2.0

rA_true = create_FRP2_v1.rA_true = create_FRP2_v1.kpAA_true / create_FRP2_v1.kpAB_true
rB_true = create_FRP2_v1.rB_true = create_FRP2_v1.kpBB_true / create_FRP2_v1.kpBA_true
rX_true = create_FRP2_v1.rX_true = create_FRP2_v1.kpAA_true / create_FRP2_v1.kpBB_true

yaml_filepath = create_FRP2_v1.write_petab_files(amici_model, sbml_model_filepath)

/SBML/PyPESTO/FRP/create_FRP2_v1.py:149: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  measurement_df = pd.concat([measurement_df, df_a, df_b], ignore_index=True)
/SBML/PyPESTO/FRP/create_FRP2_v1.py:158: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  conditions_df = pd.concat([conditions_df, conditions_data], ignore_index=True)
/tmp/ipykernel_704/2633575044.py:10: DeprecationWarning: Accessing `petab.yaml` is deprecated and will be removed in the next major release. Please use `petab.v1.yaml` ins

(4.0, 20.0, 0.13333333333333333, 2.0)
(4.0, 20.0, 0.13333333333333333, 2.0)


In [7]:
import logging

import numpy as np
import petab
import amici
import amici.plotting
import sys
import os
import importlib

import pypesto
import pypesto.optimize as optimize
import pypesto.petab
import pypesto.sample as sample
import pypesto.visualize as visualize
import tempfile

# log diagnostics
logger = logging.getLogger("pypesto.sample.diagnostics")
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

# import to petab
petab_problem = petab.Problem.from_yaml(
    yaml_filepath
)
# importer = pypesto.petab.PetabImporter(petab_problem, model_name=create_FRP_v1.MODEL_NAME)

importer = pypesto.petab.PetabImporter.from_yaml(
    yaml_filepath,
    model_name=create_FRP2_v1.MODEL_NAME,
)

# Works when it runs twice for some reason
try:
    problem = importer.create_problem(force_compile=True)
except:
    problem = importer.create_problem(force_compile=True)

Visualization table not available. Skipping.
Compiling amici model to folder /SBML/amici_models/0.26.1/FRP2_v1.
Compiling amici model to folder /SBML/amici_models/0.26.1/FRP2_v1.
2024-08-22 23:20:14.610 - amici.petab.sbml_import - INFO - Importing model ...
2024-08-22 23:20:14.611 - amici.petab.sbml_import - INFO - Validating PEtab problem ...
Visualization table not available. Skipping.
2024-08-22 23:20:14.620 - amici.petab.sbml_import - INFO - Model name is 'FRP2_v1'.
Writing model code to '/SBML/amici_models/0.26.1/FRP2_v1'.
2024-08-22 23:20:14.621 - amici.petab.sbml_import - INFO - Species: 10
2024-08-22 23:20:14.621 - amici.petab.sbml_import - INFO - Global parameters: 9
2024-08-22 23:20:14.622 - amici.petab.sbml_import - INFO - Reactions: 15
2024-08-22 23:20:14.628 - amici.sbml_import - DEBUG - Finished validating SBML                    ++ (5.24E-04s)
2024-08-22 23:20:14.632 - amici.sbml_import - DEBUG - Finished converting SBML local parameters   ++ (1.89E-04s)
2024-08-22 23:20

running build_ext
------------------------------ model_ext ------------------------------
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test CUR_FLAG_SUPPORTED
-- Performing Test CUR_FLAG_SUPPORTED - Success
-- Performing Test CUR_FLAG_SUPPORTED
-- Performing Test CUR_FLAG_SUPPORTED - Success
-- Performing Test CUR_FLAG_SUPPORTED
-- Performing Test CUR_FLAG_SUPPORTED - Success
-- Performing Test CUR_FLAG_SUPPORTED
-- Performing Test CUR_FLAG_SUPPORTED - Success
-- Appending flags from AMICI_CXXFLAGS: -fopenmp
-- Appending

In [5]:
sampler = sample.AdaptiveParallelTemperingSampler(
    internal_sampler=sample.AdaptiveMetropolisSampler(), n_chains=3,
)

Initializing betas with "near-exponential decay".


In [6]:
%%capture
n_samples = 10_000
result = sample.sample(
    problem,
    n_samples=n_samples,
    sampler=sampler,
    x0=np.array([
        np.log(1/rA_true), 
        np.log(1/rB_true), 
        np.log(1/rX_true), 
        np.log(1/kpAA_true)]),
    filename=None,
)
elapsed_time = result.sample_result.time
print(f"Elapsed time: {round(elapsed_time,2)}")

Elapsed time: 32.023236629


In [13]:
create_FRP2_v1.save_pypesto_results(result, "test__.hdf5")

Saved optimization result to /SBML/PyPESTO/FRP/Results/FRP2_v1/test__.hdf5


In [ ]:
ax = visualize.sampling_parameter_traces(
    result, use_problem_bounds=False, size=(12, 5)
)

ax[0][0].plot([0, n_samples], np.log([rA_true, rA_true]), "k--")
ax[0][1].plot([0, n_samples], np.log([rB_true, rB_true]), "k--")
ax[1][0].plot([0, n_samples], np.log([rX_true, rX_true]), "k--")
ax[1][1].plot([0, n_samples], np.log([kpAA_true, kpAA_true]), "k--")


In [ ]:
sample.geweke_test(result=result)
ax = visualize.sampling_parameter_traces(
    result, use_problem_bounds=False, size=(12, 5)
)
ax[0][0].plot([0, n_samples], np.log([rA_true, rA_true]), "k--")
ax[0][1].plot([0, n_samples], np.log([rB_true, rB_true]), "k--")
ax[1][0].plot([0, n_samples], np.log([rX_true, rX_true]), "k--")
ax[1][1].plot([0, n_samples], np.log([kpAA_true, kpAA_true]), "k--")

In [ ]:
ax = visualize.sampling_parameter_traces(
    result, use_problem_bounds=False, full_trace=True, size=(10, 4)
)
ax[0][0].plot([0, n_samples], np.log([rA_true, rA_true]), "k--")
ax[0][1].plot([0, n_samples], np.log([rB_true, rB_true]), "k--")
ax[1][0].plot([0, n_samples], np.log([rX_true, rX_true]), "k--")
ax[1][1].plot([0, n_samples], np.log([kpAA_true, kpAA_true]), "k--")

In [ ]:
alpha = [99, 95, 90]
import matplotlib.pyplot as plt
ax = visualize.sampling_parameter_cis(result, alpha=alpha, size=(10, 5))
line1, = ax.plot([np.log(rA_true)], 0, 'o', color='tab:red')
line2, = ax.plot([np.log(rB_true)], 1, 'o', color='tab:red')
line3, = ax.plot([np.log(rX_true)], 2, 'o', color='tab:red')
line4, = ax.plot([np.log(kpAA_true)], 3, 'o', color='tab:red')

# Get existing legend handles and labels
handles, labels = ax.get_legend_handles_labels()

unique_handles_labels = list(dict(zip(labels, handles)).items())
unique_labels, unique_handles = zip(*unique_handles_labels)

# Update the legend with unique handles and labels
ax.legend(unique_handles, unique_labels)
ax.set_yticks([0,1,2,3],['$r_{A}$', '$r_{B}$', '$r_{X}$', '$k_{pAA}$'])
ax.set_xlabel('parameter value')
ax.set_ylabel('')
ax.set_xlim([-5, 5])
plt.show()

In [ ]:
ci_level = 0.6827
np.exp(sample.calculate_ci_mcmc_sample(result, ci_level=ci_level))

In [ ]:
for i_chain in range(len(result.sample_result.betas)):
    ax = visualize.sampling_1d_marginals(
        result, i_chain=i_chain, suptitle=f"Chain: {i_chain}"
    )
    
    y_max_00 = ax[0][0].get_ylim()[1]
    y_max_01 = ax[0][1].get_ylim()[1]
    y_max_10 = ax[1][0].get_ylim()[1]
    y_max_11 = ax[1][1].get_ylim()[1]
    
    ax[0][0].vlines(np.log(rA_true), 0, y_max_00, color="tab:red", linestyle="--")
    ax[0][1].vlines(np.log(rB_true), 0, y_max_01, color="tab:red", linestyle="--")
    ax[1][0].vlines(np.log(rX_true), 0, y_max_10, color="tab:red", linestyle="--")
    ax[1][1].vlines(np.log(kpAA_true), 0, y_max_11, color="tab:red", linestyle="--")

In [ ]:
petab_problem = petab.Problem.from_yaml(
    yaml_filepath
)
petab_problem.measurement_df


In [ ]:
from pypesto.C import AMICI_STATUS, AMICI_T, AMICI_X, AMICI_Y
from pypesto.predict import AmiciPredictor


# This post_processor will transform the output of the simulation tool
# such that the output is compatible with the next steps.
def post_processor(amici_outputs, output_type, output_ids):
    outputs = [
        (
            amici_output[output_type]
            if amici_output[AMICI_STATUS] == 0
            else np.full((len(amici_output[AMICI_T]), len(output_ids)), np.nan)
        )
        for amici_output in amici_outputs
    ]
    return outputs


# Setup post-processors for both states and observables.
from functools import partial

amici_objective = result.problem.objective
state_ids = amici_objective.amici_model.getStateIds()
observable_ids = amici_objective.amici_model.getObservableIds()
post_processor_x = partial(
    post_processor,
    output_type=AMICI_X,
    output_ids=state_ids,
)
post_processor_y = partial(
    post_processor,
    output_type=AMICI_Y,
    output_ids=observable_ids,
)

# Create pyPESTO predictors for states and observables
predictor_x = AmiciPredictor(
    amici_objective,
    post_processor=post_processor_x,
    output_ids=state_ids,
)
predictor_y = AmiciPredictor(
    amici_objective,
    post_processor=post_processor_y,
    output_ids=observable_ids,
)

from pypesto.C import EnsembleType
from pypesto.ensemble import Ensemble

# corresponds to only the estimated parameters
x_names = result.problem.get_reduced_vector(result.problem.x_names)

# Create the ensemble with the MCMC chain from parallel tempering with the real temperature.
ensemble = Ensemble.from_sample(
    result,
    chain_slice=slice(
        None, None, 5
    ),  # Optional argument: only use every fifth vector in the chain.
    x_names=x_names,
    ensemble_type=EnsembleType.sample,
    lower_bound=result.problem.lb,
    upper_bound=result.problem.ub,
)

from pypesto.engine import MultiProcessEngine

engine = MultiProcessEngine()

ensemble_prediction = ensemble.predict(
    predictor_x, prediction_id=AMICI_X, engine=engine
)
from pypesto.C import CONDITION, OUTPUT

credibility_interval_levels = [90, 95, 99]

ax = visualize.sampling_prediction_trajectories(
    ensemble_prediction,
    levels=credibility_interval_levels,
    size=(10, 5),
    axis_label_padding=60,
    groupby=CONDITION,
)

In [ ]:
ax = visualize.sampling_prediction_trajectories(
    ensemble_prediction,
    levels=credibility_interval_levels,
    size=(20, 10),
    axis_label_padding=60,
    groupby=OUTPUT,
)

In [ ]:
mdf = petab_problem.measurement_df
# Create a custom objective with new output timepoints.
num_conditions=2
timepoints = [np.linspace(0, 100, 100) for i in range(num_conditions)]
amici_objective_custom = amici_objective.set_custom_timepoints(
    timepoints=timepoints
)

# Create an observable predictor with the custom objective.
predictor_y_custom = AmiciPredictor(
    amici_objective_custom,
    post_processor=post_processor_y,
    output_ids=observable_ids,
    condition_ids=[edata.id for edata in amici_objective_custom.edatas],
)

# Predict then plot.
ensemble_prediction = ensemble.predict(
    predictor_y_custom, prediction_id=AMICI_Y, engine=engine
)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax = visualize.sampling_prediction_trajectories(
    ensemble_prediction,
    levels=credibility_interval_levels,
    groupby=CONDITION,
    measurement_df=mdf,
    axes=ax,
)
# plt.tight_layout()

# ax[0][0].set_title('')
# ax[1][0].set_title('')
# ax[0][1].set_title('')
# ax[1][1].set_title('')
ax[0].set_ylim([-0.1, 2.5])
ax[1].set_ylim([-0.1, 2.5])
# ax[2].set_ylim([-0.1, 2.5])
# fig.set_title('')
ax[0].set_xlabel('Time (s)')
ax[1].set_xlabel('Time (s)')
# ax[2].set_xlabel('Time (s)')
# ax[0].set_ylabel('Simulated Concentration (M)')
# ax[0][0].set_ylim([-0.1, 2.1])
# ax[1][0].set_ylim([-0.1, 2.1])
# ax[0][1].set_ylim([-0.1, 2.1])
# ax[1][1].set_ylim([-0.1, 2.1])

plt.show()